In [1]:
import time
import os
import json
import pandas as pd
import numpy as np
from datetime import date
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
# PATHs
# The directory where the raw transaction and punk data are stored
ORI_DATA_PATH = '../data/ori'

# The directory where the databases are stored
DATABASE_PATH = '../data/database'
if os.path.exists(DATABASE_PATH) is False:
    os.makedirs(DATABASE_PATH)


## Database

In [3]:
def create_punk_db(CSV_PATH=ORI_DATA_PATH):
    punk_db = pd.read_csv('{}/{}'.format(CSV_PATH, 'punk_info.csv'))

    # rename columns
    punk_db.columns = ['punk_id', 'type', 'gender',
                       'skin_tone', 'attr_count', 'attributes']

    # strip type, gender, skin_tone
    punk_db['type'] = punk_db['type'].apply(lambda x: x.strip())
    punk_db['gender'] = punk_db['gender'].apply(lambda x: x.strip())
    punk_db['skin_tone'] = punk_db['skin_tone'].apply(lambda x: x.strip())
    punk_db['skin_tone'] = punk_db['skin_tone'].apply(
        lambda x: 'Non-human' if x == '' else x)
    
    # set skin_tone color
    skin_tones = ['Medium', 'Dark', 'Light', 'Albino', 'Non-human']
    colors = ['#DB9065', '#A4031F', '#F2A359', '#F2DC5D', '#8DFFCD']
    color_by_skin_tone = dict(zip(skin_tones, colors))
    
    punk_db['skin_tone_color'] = punk_db['skin_tone'].apply(lambda x: color_by_skin_tone[x])

    # make attributes as list
    punk_db['attributes'] = punk_db['attributes'].apply(
        lambda x: [i.strip() for i in x.split('/')])

    # set punk image url
    punk_db['img_url'] = punk_db['punk_id'].apply(
        lambda x: 'https://www.larvalabs.com/cryptopunks/cryptopunk{}.png'.format(x))

    # set punk_id as index
    punk_db.set_index('punk_id', inplace=True)

    return punk_db


punk_db = create_punk_db(ORI_DATA_PATH)
punk_db.to_csv('{}/punk_db.csv'.format(DATABASE_PATH), index=True)
print('Cryptopunk database saved to {}/punk_db.csv'.format(DATABASE_PATH))
punk_db

Cryptopunk database saved to ../data/database/punk_db.csv


,type,gender,skin_tone,attr_count,attributes,skin_tone_color,img_url
punk_id,,,,,,,
0,Human,Female,Medium,3,"[Green Eye Shadow, Earring, Blonde Bob]",#DB9065,https://www.larvalabs.com/cryptopunks/cryptopu...
1,Human,Male,Dark,2,"[Smile, Mohawk]",#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...
2,Human,Female,Light,1,[Wild Hair],#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...
3,Human,Male,Dark,3,"[Wild Hair, Nerd Glasses, Pipe]",#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...
4,Human,Male,Medium,4,"[Big Shades, Wild Hair, Earring, Goat]",#DB9065,https://www.larvalabs.com/cryptopunks/cryptopu...
...,...,...,...,...,...,...,...
9995,Human,Female,Albino,2,"[Purple Eye Shadow, Straight Hair Dark]",#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...
9996,Human,Male,Light,4,"[Cigarette, Earring, Crazy Hair, Smile]",#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...
9997,Zombie,Male,Non-human,2,"[Front Beard, Cap Forward]",#8DFFCD,https://www.larvalabs.com/cryptopunks/cryptopu...


In [6]:
today = '2022-07-27' #str(date.today())

dune_dict = json.load(
    open('{}/punkBought_{}.json'.format(ORI_DATA_PATH, today)))

In [7]:
def index_trader(buyer, seller):
    trader = list(set(list(buyer.unique()) + list(seller.unique())))
    return {trader[i]: i for i in range(len(trader))}


def create_tx_db(dune_dict):
    temp_idx = 0
    # punkBought data frame
    bt_data = dict()
    for year in dune_dict:
        data_list = dune_dict[year]['data']['get_result_by_result_id']
        bt_data.update({temp_idx+i: data_list[i]['data']
                       for i in range(len(data_list))})
        temp_idx += len(data_list)

    # concatenate data frames
    tx_db = pd.DataFrame.from_dict(bt_data, orient='index')

    # remove the wrong tx
    tx_db = tx_db[tx_db['eth_price'] < 10000]

    # set date_time
    tx_db['date'] = pd.to_datetime(tx_db['_date'])

    # remove txs whose buyer is \x0000000000000000000000000000000000000000
    print("Number of tx whose buyer is x0: {}".format(len(tx_db[tx_db['buyer'] ==
                  '\\x0000000000000000000000000000000000000000'])))
    tx_db = tx_db[tx_db['buyer'] !=
                  '\\x0000000000000000000000000000000000000000']

    # reindex seller and buyer using trader_id
    trader_index_dict = index_trader(tx_db['buyer'], tx_db['seller'])
    tx_db['from'] = tx_db['seller'].apply(lambda x: trader_index_dict[x])
    tx_db['to'] = tx_db['buyer'].apply(lambda x: trader_index_dict[x])

    # sort by date_time
    tx_db.sort_values(by='date', inplace=True)

    # index transaction
    tx_db['tx_id'] = range(len(tx_db))
    tx_db = tx_db.loc[:, ['tx_id', 'date',
                          'from', 'to', 'eth_price', 'punk_id']]
    tx_db.set_index('tx_id', inplace=True)

    
    print('Total {} transactions'.format(len(tx_db)))
    print('Total {} unique traders'.format(len(trader_index_dict)))
    print('Total {} unique punk_id'.format(len(tx_db['punk_id'].unique())))

    return tx_db, trader_index_dict


tx_db, trader_index_dict = create_tx_db(dune_dict)
tx_db = tx_db.merge(punk_db.reset_index(), on='punk_id')
tx_db.to_csv('{}/tx_db.csv'.format(DATABASE_PATH), index=True)
print('Transaction database saved to {}/tx_db.csv'.format(DATABASE_PATH))
tx_db

Number of tx whose buyer is x0: 3358
Total 17826 transactions
Total 6438 unique traders
Total 6275 unique punk_id
Transaction database saved to ../data/database/tx_db.csv


,date,from,to,eth_price,punk_id,type,gender,skin_tone,attr_count,attributes,skin_tone_color,img_url
0,2017-06-23,1406,1218,0.03,6548,Human,Male,Albino,4,"[Front Beard, Earring, Do-rag, Clown Eyes Green]",#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...
1,2017-07-03,1218,3828,0.49,6548,Human,Male,Albino,4,"[Front Beard, Earring, Do-rag, Clown Eyes Green]",#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...
2,2017-06-23,1406,1218,0.04,5719,Human,Female,Medium,3,"[Welding Goggles, Dark Hair, Cigarette]",#DB9065,https://www.larvalabs.com/cryptopunks/cryptopu...
3,2019-03-20,920,1472,0.99,5719,Human,Female,Medium,3,"[Welding Goggles, Dark Hair, Cigarette]",#DB9065,https://www.larvalabs.com/cryptopunks/cryptopu...
4,2017-06-23,1406,2518,0.25,1841,Human,Female,Dark,3,"[Black Lipstick, Green Eye Shadow, Mohawk Thin]",#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...
...,...,...,...,...,...,...,...,...,...,...,...,...
17821,2022-07-17,2292,2070,88.88,4656,Human,Female,Medium,2,"[Blonde Short, Classic Shades]",#DB9065,https://www.larvalabs.com/cryptopunks/cryptopu...
17822,2022-07-17,5922,916,88.00,4749,Human,Male,Medium,3,"[Frown, Do-rag, Small Shades]",#DB9065,https://www.larvalabs.com/cryptopunks/cryptopu...
17823,2022-07-18,107,1044,84.00,4569,Human,Male,Light,3,"[Clown Eyes Blue, Shadow Beard, Mohawk]",#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...
17824,2022-07-21,244,801,86.68,4755,Human,Male,Dark,3,"[Messy Hair, Earring, Normal Beard]",#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...


In [11]:
id_address_dict = {v: k for k, v in trader_index_dict.items()}
with open('{}/addresses.json'.format(DATABASE_PATH), "w") as outfile:
    json.dump(id_address_dict, outfile)

In [13]:
import collections

trader_ids = sorted(list(tx_db['from']) + list(tx_db['to']))
print(len(trader_ids))

counter=collections.Counter(trader_ids)
addresses_count = dict(counter)

35652


In [25]:
{k: v for k, v in sorted(addresses_count.items(), key=lambda item: item[1], reverse=True)}

{2172: 997,
 405: 768,
 802: 600,
 6308: 396,
 199: 371,
 2904: 303,
 2207: 266,
 3072: 247,
 15: 236,
 6383: 196,
 5497: 192,
 674: 184,
 4956: 177,
 627: 166,
 2248: 154,
 2051: 149,
 3122: 147,
 6141: 145,
 336: 139,
 596: 136,
 1837: 135,
 2649: 135,
 2932: 132,
 5191: 132,
 6101: 132,
 5978: 130,
 614: 126,
 942: 125,
 901: 122,
 3256: 121,
 2569: 120,
 6367: 117,
 1222: 106,
 2609: 106,
 2953: 106,
 5734: 106,
 3138: 105,
 3994: 105,
 566: 104,
 4448: 103,
 2767: 99,
 4611: 99,
 6061: 98,
 1391: 95,
 5095: 95,
 900: 92,
 4401: 92,
 621: 91,
 3335: 91,
 3230: 89,
 4221: 89,
 1358: 88,
 6207: 87,
 352: 85,
 2550: 85,
 4601: 83,
 5779: 83,
 350: 82,
 2977: 82,
 3009: 82,
 3531: 79,
 4769: 78,
 3546: 74,
 363: 73,
 5083: 73,
 5103: 73,
 5405: 72,
 6164: 72,
 1566: 71,
 1582: 71,
 3839: 71,
 2070: 70,
 3465: 70,
 526: 68,
 2078: 67,
 5970: 67,
 1425: 65,
 3960: 65,
 707: 64,
 5893: 64,
 4344: 63,
 1616: 62,
 2615: 62,
 4575: 62,
 1288: 61,
 5945: 60,
 371: 59,
 5215: 59,
 5613: 58,
 5

In [33]:
id_address_dict[2904]

'\\x577ebc5de943e35cdf9ecb5bbe1f7d7cb6c7c647'

### Top addresses:

| address                                   | trader_id | # of tx | who                    |
|-------------------------------------------|-----------|---------|------------------------|
| x1919db36ca2fa2e15f9000fd9cdc2edcf863e685 | 2172      | 997     | Punks OTC              |
| x53ede7cae3eb6a7d11429fe589c0278c9acbe21a | 405       | 768     | Rarible: Braithwaite   |
| xd387a6e4e84a6c86bd90c158c6028a58cc8ac459 | 802       | 600     | Pranksy                |
| x00d7c902fbbcd3c9db2da80a439c94486c50eb81 | 6308      | 396     |                        |
| x269616d549d7e8eaa82dfb17028d0b212d11232a | 199       | 371     | BeaconProxy (Contract) |
| x577ebc5de943e35cdf9ecb5bbe1f7d7cb6c7c647 | 2904      | 303     |                        |